In [1]:
import sys
sys.path.append('/Users/yuhangzhan/Git/statement_reader')

In [2]:
from boa import BOA_CREDIT

In [3]:
fpath = '/Users/yuhangzhan/Git/statement_reader/data/pdf/boa/credit/eStmt_2018-01-28.pdf'

In [4]:
boa = BOA_CREDIT(fpath)

-2019-05-14 23:37:04,060-BOA-INFO: BOA credit job start
-2019-05-14 23:37:04,265-main-INFO: Begin to execute function: get_credit_statement_info
-2019-05-14 23:37:04,274-main-INFO: Finish executing function: get_credit_statement_info
-2019-05-14 23:37:04,274-BOA-INFO: BOA credit job finished


In [5]:
boa.summary

{'previous_balance': 291.22,
 'payments_and_other_credits': -1458.05,
 'fees_charged': 0.0,
 'interest_charged': 0.0,
 'new_balance_total': 686.21}

## import conf

In [10]:
import json

In [41]:
personal_conf_path = '/Users/yuhangzhan/Git/statement_reader/conf/personal_category_config.json'
default_conf_path = '/Users/yuhangzhan/Git/statement_reader/conf/db_default_config.json'
def import_conf(file_path):
    ''' read configuration json file, return as dict'''
    with open(file_path, 'r') as f:
        raw_conf = ''.join([line for line in f.readlines() if not line.strip().startswith('//')])
        json_conf = json.loads(raw_conf)
    conf = {key:[(int(idx), *[val if val else 'Null' for _, val in fields.items()]) 
                  for idx, fields in value.items() ]
           for key, value in json_conf.items()}
    return conf

In [84]:
personal_conf = import_conf(personal_conf_path)
default_conf = import_conf(default_conf_path)

In [85]:
conf = {**default_conf, **personal_conf}

In [86]:
conf.keys()

dict_keys(['bank', 'account_type', 'card_type', 'transaction_type', 'personal_category'])

## import insert conf sql

In [108]:
insert_conf_sql_path = '/Users/yuhangzhan/Git/statement_reader/src/database/insert_init_setting.sql'
def import_conf_query(file_path):
    '''read insert conf sql file, return as dict'''
    with open(file_path, 'r') as f:
        raw_sql = ''.join([line for line in f.readlines() if not line.strip().startswith('--') and line.strip()])
    splited_sql = [query for query in raw_sql.split(';') if query]
    dict_sql = [(query.split()[2], query.strip()+';') for query in splited_sql]
    return dict(dict_sql)

In [109]:
insert_query = import_conf_query(insert_conf_sql_path)

## Connect db

In [90]:
import sqlite3

In [102]:
conn = sqlite3.connect('/Users/yuhangzhan/Git/statement_reader/data/db/billing.db')
cursor = conn.cursor()

In [103]:
cursor.executemany("INSERT INTO bank (bank_id, bank_name) values (?, ?);", [(1, 'boa'), (2,'chase')])
conn.commit()

## insert conf

In [110]:
for table, query in insert_query.items():
    print(table, query, conf[table])

bank INSERT INTO bank (bank_id, bank_name) values (?, ?); [(1, 'boa'), (2, 'chase')]
account_type INSERT INTO account_type (account_type_id, DESCRIPTION) values (?, ?); [(1, 'checking'), (2, 'saving')]
card_type INSERT INTO card_type (card_type_id, DESCRIPTION) values (?, ?); [(1, 'debit'), (2, 'credit')]
transaction_type INSERT INTO transaction_type (transaction_type_id, bank_id, DESCRIPTION) values (?, ?, ?); [(1, 1, 'payment'), (2, 1, 'purchase'), (3, 1, 'fee'), (4, 1, 'interest')]
personal_category INSERT INTO personal_category (p_category_id, DESCRIPTION, COMMENT) values (?, ?, ?); [(1, 'deposit', 'transfer, deposit, wire transfer'), (2, 'study', 'Null'), (3, 'food', 'resturuant, cafe'), (4, 'shopping', 'Null'), (5, 'commute', 'parking, garage, gas, car insurance, ticket'), (6, 'living', 'gorcery, healcare'), (7, 'housing', 'rantal, utility, apt insurance'), (8, 'travel', 'Null'), (9, 'car', 'Null'), (10, 'oudoor', 'equipment, traning, membership fee')]


In [107]:
insert_query

{'(bank_id,': 'INSERT INTO bank (bank_id, bank_name) values (?, ?);',
 '(account_type_id,': 'INSERT INTO account_type (account_type_id, DESCRIPTION) values (?, ?);',
 '(card_type_id,': 'INSERT INTO card_type (card_type_id, DESCRIPTION) values (?, ?);',
 '(transaction_type_id,': 'INSERT INTO transaction_type (transaction_type_id, bank_id, DESCRIPTION) values (?, ?, ?);',
 '(p_category_id,': 'INSERT INTO personal_category (p_category_id, DESCRIPTION, COMMENT) values (?, ?, ?);'}

## insert trans

In [6]:
boa.statement_info['purchase']

[{'types': 'purchase',
  'transaction_date': '2018-01-07',
  'posting_date': '2018-01-08',
  'reference_number': 1612,
  'account_number': 7577,
  'amount': 19.99,
  'description': 'NINTENDO OF AMERICA-US 800-255-3700 WA'},
 {'types': 'purchase',
  'transaction_date': '2018-01-16',
  'posting_date': '2018-01-17',
  'reference_number': 1956,
  'account_number': 7577,
  'amount': 4.73,
  'description': 'FOXNEWSST2513          DALLAS      TX'},
 {'types': 'purchase',
  'transaction_date': '2018-01-17',
  'posting_date': '2018-01-18',
  'reference_number': 4088,
  'account_number': 7577,
  'amount': 106.52,
  'description': 'AMAZON MKTPLACE PMTS        AMZN.COM/BILLWA'},
 {'types': 'purchase',
  'transaction_date': '2018-01-17',
  'posting_date': '2018-01-18',
  'reference_number': 1242,
  'account_number': 7577,
  'amount': 62.96,
  'description': 'AMAZON MKTPLACE PMTS WWW.WWW.AMAZON.COWA'},
 {'types': 'purchase',
  'transaction_date': '2018-01-18',
  'posting_date': '2018-01-18',
  'refe